In [1]:
import pandas as pd
import graphviz
import pydotplus
from sklearn import tree
from sklearn.externals.six import StringIO
from graphviz import Digraph
import pydotplus
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
filename = 'd41d8cd9.xlsx'
fd = pd.read_excel(filename)
fd.columns

Index(['カテゴリー', '品種', 'ブロック', '株番号', '肥大部長(20%)', '肥大部長(思う長さ)', '肥大部長(1cmまで)',
       '最大幅', '最大幅20%', 'Unnamed: 9', '長幅比(20%)', '長幅比(思う)', '長幅比(1cm)',
       'Unnamed: 13', 'fit1(20%)', 'fit2(20%)', 'fit1(先端まで)', 'fit2(先端まで)',
       'Unnamed: 18', '最大幅位置', '膨らみの位置(20%)', '膨らみの位置(思う)', '膨らみの位置(1cm)',
       'Unnamed: 23', '首径', '膨らみの程度', 'Unnamed: 26', '幅の一定さ1', '幅の一定さ2',
       '膨らみの程度(5%)', '膨らみの程度(10%)', 'Unnamed: 31', '曲がり', 'Unnamed: 33',
       '5形質(幅の一定さ(方法1))', '6形質(膨らみの程度(10%)+幅の一定さ(方法1))'],
      dtype='object')

In [3]:
X = fd[['長幅比(20%)','膨らみの位置(20%)','膨らみの程度(10%)','幅の一定さ1','fit1(20%)','曲がり']]

In [4]:
# 決定木を準備
classifier = tree.DecisionTreeClassifier()
# 訓練データもとに決定木を構成する
classifier.fit(X,fd['カテゴリー'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [5]:
# 完成した決定木をPDFで出力
dot_data = StringIO() # 仮想テキストファイル
tree.export_graphviz(classifier, out_file=dot_data) # 決定木の描画用データ dot_data を生成する
dot_text = dot_data.getvalue() # dot_data を文字列として取り出し， X[_] を表見出しに使っている言葉で置換
dot_text = dot_text.replace('X[0]','長幅比(20%)').replace('X[1]','膨らみの位置(20%)').replace('X[2]','膨らみの程度(10%)').replace('X[3]','幅の一定さ1').replace('X[4]','fit1(20%)').replace('X[5]','曲がり')
# 決定木を PDF ファイルとして出力
graph = pydotplus.graph_from_dot_data(dot_text)

In [6]:
graph.write_pdf('daikon.pdf')
#graph.write_png(ファイル名)  png 画像として書き出し

True

In [7]:
result=classifier.predict(X)

In [8]:
result

array(['円筒', '円筒', '円筒', '円筒', '円筒', '円筒', '円筒', '円筒', '円筒', '楕円', '楕円',
       '楕円', '楕円', '楕円', '楕円', '楕円', '楕円', '楕円', '逆三角', '逆三角', '逆三角',
       '逆三角', '逆三角', '逆三角', '逆三角', '逆三角', '逆三角', '丸', '丸', '丸', '丸', '丸',
       '丸', '丸', '丸', '丸', '横楕円', '横楕円', '横楕円', '横楕円', '横楕円', '横楕円',
       '横楕円', '横楕円', '横楕円', '角', '角', '角', '角', '角', '角', '角', '角', '角',
       '角', '角', 'だるま', 'だるま', 'だるま', '円筒', 'だるま', 'だるま', 'だるま', 'だるま',
       'だるま', '先流れ', '先流れ', '先流れ', '横楕円', '横楕円', '丸', '横楕円', '丸', '角',
       '三角', '角', '角', '三角', '三角', '角', '角', '三角', '三角', '三角', '丸', '横楕円',
       '横楕円', '横楕円', '横楕円', '円筒', '楕円', '丸', '円筒', '丸', '丸', '横楕円', '横楕円',
       '逆三角', '逆三角', '楕円', '楕円', '楕円', '楕円', '楕円', '丸', '楕円', '先流れ', '円筒',
       '逆三角', '円筒', '逆三角', '逆三角', '逆三角', '逆三角', '先流れ', '円筒', '円筒', '逆三角',
       'だるま', '角', '三角', '三角', '三角', '先流れ', '先流れ', '先流れ', '楕円', '丸',
       '先流れ', '角', '丸', '円筒', '逆三角', '円筒', '楕円', '先流れ', '先流れ', '先流れ',
       '先流れ', '先流れ', '先流れ', '先流れ', '先流れ', '先流れ', '三角', '三角',

In [9]:
# 誤った判定結果があるかどうか？
print(len(fd[fd['カテゴリー']!=result]))

0


## leave-one-out

In [10]:
classifier = tree.DecisionTreeClassifier()
print("誤判定のケースだけ書き出し")
for i in range(len(fd)):
    # 訓練データもとに決定木を構成する
    classifier.fit(pd.concat([X[0:i],X[i+1:]]), pd.concat([fd['カテゴリー'][0:i],fd['カテゴリー'][i+1:]]))
    if fd['カテゴリー'][i] != classifier.predict(X[i:i+1])[0]:
        print('{}  正解{} - 判定{}'.format(i,fd['カテゴリー'][i],classifier.predict(X[i:i+1])[0]))

誤判定のケースだけ書き出し
1  正解円筒 - 判定楕円
6  正解円筒 - 判定楕円
9  正解楕円 - 判定逆三角
17  正解楕円 - 判定円筒
25  正解逆三角 - 判定円筒
30  正解丸 - 判定楕円
52  正解角 - 判定三角
56  正解だるま - 判定逆三角
70  正解丸 - 判定楕円
78  正解三角 - 判定先流れ
83  正解三角 - 判定角
90  正解楕円 - 判定丸
97  正解逆三角 - 判定楕円
99  正解楕円 - 判定先流れ
101  正解楕円 - 判定円筒
106  正解先流れ - 判定楕円
109  正解円筒 - 判定先流れ
111  正解逆三角 - 判定円筒
112  正解逆三角 - 判定楕円
113  正解逆三角 - 判定だるま
120  正解三角 - 判定先流れ
125  正解先流れ - 判定三角
130  正解丸 - 判定横楕円
131  正解円筒 - 判定だるま
133  正解円筒 - 判定逆三角
145  正解三角 - 判定角


In [11]:
# 前から120データで学習して、残り３６で検証
classifier.fit(X[0:120], fd['カテゴリー'][0:120])
print("誤判定のケースだけ書き出し")
for i in range(121,len(fd)):
    if fd['カテゴリー'][i] != classifier.predict(X[i:i+1])[0]:
        print('{}  正解 {} - 判定 {}'.format(i,fd['カテゴリー'][i],classifier.predict(X[i:i+1])[0]))

誤判定のケースだけ書き出し
125  正解 先流れ - 判定 三角
130  正解 丸 - 判定 横楕円
131  正解 円筒 - 判定 だるま
132  正解 逆三角 - 判定 楕円
133  正解 円筒 - 判定 逆三角
134  正解 楕円 - 判定 逆三角
139  正解 先流れ - 判定 楕円
